# Algorithm for the Addition of Two Points: $P + Q$
- If $P = O$, then $P + Q = Q$.
- Otherwise, if $Q = O$, then $P + Q = P$.
- Otherwise, let $P = (x_1, y_1)$ and $Q = (x_2, y_2)$.
- If $x_1 = x_2$ and $y_1 = -y_2$, then $P + Q = O$.
- Otherwise:
  - If $P \neq Q$, then $\lambda = (y_2 - y_1) / (x_2 - x_1)$.
  - If $P = Q$, then $\lambda = (3x_1^2 + a) / 2y_1$.
- $x_3 = \lambda^2 - x_1 - x_2$.
- $y_3 = \lambda(x_1 - x_3) - y_1$.
- $P + Q = (x_3, y_3)$.

In [91]:
# imports:
from typing import List
from dataclasses import dataclass
from hashlib import sha1

In [92]:
# placeholder for inf
O = [-1, -1]

@dataclass
class EllipticCurve:
    a: int
    b: int
    p: int

In [93]:
def equals(P: List[int], Q: List[int]):
    return all([a == b for a, b in zip(P, Q)])

In [94]:
def extended_gcd(a, b):
    if a == 0:
        return (b, 0, 1)
    
    g, x1, y1 = extended_gcd(b % a, a)
    
    x = y1 - (b // a) * x1
    y = x1
    
    return (g, x, y)

In [95]:
def inv(x: int, p: int):
    t, s, _ = extended_gcd(x, p)
    
    if t != 1:
        print(f"Invalid input {x}")
        raise Exception("No inverse")
    else:
        return s % p

In [96]:
def sub(x: int, y: int, p: int):
    res = x - y
    if res < 0:
        return res + p
    
    return res % p

In [97]:
def point_addition(P: List[int], Q: List[int], ec: EllipticCurve) -> List[int]:
    if equals(P, O):
        return Q
    
    if equals(Q, O):
        return P
    
    x_1, y_1 = P
    x_2, y_2 = Q

    if x_1 == x_2 and y_1 == -y_2:
        return O
    
    if not equals(P, Q):
        l = sub(y_2,  y_1, ec.p) * inv(sub(x_2, x_1, ec.p), ec.p)

    else:
        l = (3 * (x_1 ** 2) + ec.a) * inv(2 * y_1, ec.p)

    l = l % ec.p
    
    x_3 = sub(sub(l ** 2, x_1, ec.p), x_2, ec.p)
    y_3 = sub(l * sub(x_1, x_3, ec.p), y_1, ec.p)

    return [x_3, y_3]
    
    

💡 You can test your algorithm by asserting: X+Y=(1024,4440) and X+X=(7284,2107) for X=(5274,2841) and Y=(8669,740).

The algorithm is based on the elliptic curve E: Y2=X3+497X+1768 mod 9739.

In [98]:
X=[5274,2841]
Y=[8669,740]

# the elliptic curve E: Y2 = X3 + 497X + 1768 mod 9739
ec = EllipticCurve( a=497, b=1768, p=9739)

print(\
    f"""
    expected values be:
    X+Y=[1024,4440] and X+X=[7284,2107]
    result values:
    X+Y={point_addition(X, Y, ec)} and X+X={point_addition(X, X, ec)}
    """)


    expected values be:
    X+Y=[1024,4440] and X+X=[7284,2107]
    result values:
    X+Y=[1024, 4440] and X+X=[7284, 2107]
    


Using the above curve, and the points P=(493,5564), Q=(1539,4742), R=(4403,5202), find the point S(x,y)=P+P+Q+R by implementing the above algorithm.

In [99]:
P=[493,5564]
Q=[1539,4742]
R=[4403,5202]

# S(x,y)=P+P+Q+R
res = point_addition(P, P, ec)
res = point_addition(res, Q, ec)
S = point_addition(res, R, ec)

print(f"crypto{S}")

crypto[4215, 2162]


# Double and Add Algorithm for Scalar Multiplication

**Input:**  
$P \in E(\mathbb{F}_p)$ and an integer $n > 0$.

**Output:**  
$Q = [n]P \in E(\mathbb{F}_p)$.

---

## Algorithm (Double-and-Add)

1. Initialize: $Q = P$ and $R = O$.
2. While $n > 0$:
   1. If $n \equiv 1 \pmod{2}$, set $R = R + Q$.
   2. Set $Q = [2]Q$.
   3. Set $n = \lfloor n/2 \rfloor$.
3. Return $R$ (which equals $[n]P$).



In [100]:
def double_and_add(P: List[int], n: int, ec: EllipticCurve):
    Q = P
    R = O
    while n > 0:
        if n % 2 == 1:
            R = point_addition(R, Q, ec)
        Q = point_addition(Q, Q, ec)
        n = n // 2
    return R

In [101]:
P = [2339, 2213]
Q = double_and_add(P, 7863, ec)

print(f"crypto{Q}")

crypto[9467, 2742]


# Curves and Logs


The **Elliptic Curve Discrete Logarithm Problem (ECDLP)** is the problem of finding an integer

$$
Q = [n]P
$$

such that $Q$ is obtained by multiplying a point $P$ by some integer $n$.

Like with the **discrete logarithm problem**, scalar multiplication of a point in

$$
E(\mathbb{F}_p)
$$

is believed to be hard to invert. The most efficient algorithm runs in

$$
O(\sqrt{q})
$$

time when $P$ generates a subgroup of size $q$.

This makes it a great candidate for a **trapdoor function**.

---

## Alice & Bob’s Shared Secret

Alice and Bob want to establish a shared secret for symmetric cryptography.
They agree on:

* A curve $E$
* A prime $p$
* A generator point $G$

which generates a subgroup

$$
H = \langle G \rangle
$$

of prime order $q$.

> **Note:** In elliptic curve cryptography, it is important that the order of $G$ is prime. Constructing secure curves is non-trivial, so standardized curves are typically used.

## Elliptic Curve Diffie-Hellman (ECDH)

1. **Alice** chooses a secret random integer $n_A$ and computes

   $$
   Q_A = [n_A]G
   $$

2. **Bob** chooses a secret random integer $n_B$ and computes

   $$
   Q_B = [n_B]G
   $$

3. Alice sends $Q_A$ to Bob, and Bob sends $Q_B$ to Alice.
   An eavesdropper Eve cannot feasibly compute $n_A$ or $n_B$ due to ECDLP hardness.

4. **Alice** computes

   $$
   S = [n_A]Q_B
   $$

   **Bob** computes

   $$
   S = [n_B]Q_A
   $$

5. By associativity of scalar multiplication:

   $$
   S = [n_A]Q_B = [n_B]Q_A
   $$

   Thus, Alice and Bob share the same secret $S$.


## Example Curve

Using the given curve, prime, and generator:

$$
E: \; Y^2 = X^3 + 497X + 1768 \;\; \text{over } \mathbb{F}_{9739}
$$

Generator:

$$
G = (1804, 5368), \quad q = 9739
$$

$$
Q_A​=(815,3190)
$$

with your secret integer $n_B=1829$

In [102]:
def compute_key(q_a: List[int], n_b: int, ec: EllipticCurve):
    q_ab = double_and_add(q_a, n_b, ec)
    x_ab = q_ab[0]

    return x_ab, sha1(str(x_ab).encode()).hexdigest()

    

In [103]:

Q_A = [815,3190]
n_B = 1829

x, key = compute_key(Q_A, n_B, ec)

print(f"crypto[{key}]")

crypto[80e5212754a824d3a4aed185ace4f9cac0f908bf]


Alice and Bob are looking at the Elliptic Curve Discrete Logarithm Problem and thinking about the data they send.

They want to try and keep their data transfer as efficient as possible and realise that sending both the $x$ and $y$ coordinate of their public key isn't necessary.

As long as Alice and Bob agree on the curve parameters, there are only ever two possible values of $y$ for a given $x$.

In fact, given *either* of the values of $y$ permissible from the value $x$ they receive, the $x$ coordinate of their shared secret will be the same.

💡 For these challenges, we have used a prime $p \\equiv 3 \\pmod{4}$, which will help you find $y$ from $y^2$.

Using the curve, prime and generator:

$$E: Y^2 = X^3 + 497 X + 1768 \pmod{9739}, \quad G: (1804,5368)$$

Calculate the shared secret after Alice sends you $x(Q\_A) = 4726$, with your secret integer $n\_B = 6534$.

Use the `decrypt.py` file to decode the flag

```
{'iv': 'cd9da9f1c60925922377ea952afc212c', 'encrypted_flag': 'febcbe3a3414a730b125931dccf912d2239f3e969c4334d95ed0ec86f6449ad8'}
```

✍️You can specify which of the two possible values your public $y$ coordinate has taken by sending only one bit. Try and think about how you could do this. How are the two $y$ values related to each other?


In [ ]:
x_a = 4726
n_b = 6534

y_squared = (pow(x_a, 3, mod=ec.p) + ec.a * x_a + ec.b) % ec.p

# this works because p % 4 = 3
y_a = pow(y_squared, (ec.p + 1) // 4, mod=ec.p)

y_a_minus =  -y_a % ec.p

Q_A = [x_a, y_a]

x_ab, key = compute_key(Q_A, n_b, ec)

print(f"the first option for x value of the secret key: {x_ab}")

Q_A = [x_a, y_a_minus]

x_ab, key = compute_key(Q_A, n_b, ec)

print(f"the second option for x value of the secret key: {x_ab}")

the first option for x value of the secret key: 1791
the second option for x value of the secret key: 1791
